In [1]:
# default_exp transfer_dtsek_to_dergey

In [38]:
#export
import unicodedata
import re
import sys

from antx.utils import optimized_diff_match_patch
from tqdm.notebook import tqdm
from config import TengyurConfig, KangyurConfig
from fastcore.parallel import parallel

In [55]:
config = TengyurConfig()

In [42]:
#export
dmp = optimized_diff_match_patch()
dmp.binary_path

Path('/home/tenzin/.antx/bin/dmp')

Steps:
1. build and transfer double tsek to pedurma
1. Get dergey and pedurma double tsek text
1. Transfer pedurma double tseks to dergey base

# build and transfer double tsek pedurma

## preprocess ocr double tsek output
- remove obvious false postive double tseks (grouped double tseks, more than one)

In [61]:
#export
def remove_grouped_dtseks(text):
    return re.sub(':{3,}', '', text)

def change_dtsek_to_dollar(text):
    return re.sub(':', config.double_tsek_sym, text)

def preprocess_ocr_output(text):
    text = remove_grouped_dtseks(text)
    if config.name == "tengyur":
        text = change_dtsek_to_dollar(text)
    return text

In [62]:
assert preprocess_ocr_output('ལ་::ཁྱོལ་པོ་པ། :སྤྱི་བ་ལོ::::་ཀ') == 'ལ་$$ཁྱོལ་པོ་པ། $སྤྱི་བ་ལོ་ཀ'

In [64]:
#export
def build_pedurma_vols(replace=False):
    for vol_dir in sorted(config.pedurma_output_path.iterdir()):
        vol_fn = vol_dir.parent / f"{vol_dir.name}.txt"
        if vol_dir.is_file() or vol_dir.name == '.git': continue
        if vol_fn.is_file() and not replace: continue
        vol_text = ''
        for (i, page_fn) in enumerate(sorted(vol_dir.iterdir())):
            page_text = page_fn.read_text()
            # dont't preprocess the first page, metadata
            if i: page_text = preprocess_ocr_output(page_text)
            vol_text += page_text + "\n\n\n"
        vol_fn.write_text(vol_text)

In [65]:
build_pedurma_vols()

### transfer double tseks to pedurma

In [8]:
#export
def isNSM(char):
    # Detects nonspacing mark characters
    if unicodedata.category(char) == "Mn":
        return True
    return False

In [9]:
char = 'ཽ'
isNSM(char)

True

In [10]:
#export
def get_first_char_idx(text, char):
    """Return first char idx in `text` and -1 of not found

    found har idx is expanded for whitespces after the char.
    """
    is_char_found = False
    idx = -1
    for i in range(len(text)):
        if not is_char_found and text[i] == char:
            is_char_found = True
            idx = i
        elif is_char_found:
            if text[i] != ' ': return i-1
            elif i == len(text)-1: return i
    return idx

In [11]:
string = '01|  567 9'
idx = get_first_char_idx(string, '|')
print(repr(string[:idx+1]), repr(string[idx+1]))

string = '01|  '
idx = get_first_char_idx(string, '|')
print(repr(string[:idx+1]))

'01|  ' '5'
'01|  '


In [12]:
#export
def parse_double_tsek(text):
    double_tsek_idxs = []
    base_char_idx = 0
    for c in text:
        if c == config.double_tsek_sym:
            double_tsek_idxs.append(base_char_idx)
            continue
        base_char_idx += 1
    return double_tsek_idxs

def adjust_next_diff(i, diffs, ann_text, to_char):
    diff_mode, diff_chunk = diffs[i+1]
    # add chars till next tsek to the ann_text
    first_char_idx = get_first_char_idx(diff_chunk, to_char)
    ann_text += diff_chunk[:first_char_idx+1]
    # remove chars till next tsek from diff[i+1]
    diffs[i+1] = (diff_mode, diff_chunk[first_char_idx+1:])
    return ann_text


def transfer_dtsek(base_text, dest_text, verbose=False):
    ann_text = ''
    diffs = list(dmp.diff_main(dest_text, base_text))
    if verbose: print(diffs)
    for i in range(len(diffs)):
        if diffs[i][0] == -1:
            if config.double_tsek_sym in diffs[i][1]:
                # check for next diff adjustment
                if i < len(diffs)-1:
                    # adjust next diff[i+1] if it's first char is NSM
                    if isNSM(diffs[i+1][1][0]):
                        ann_text = adjust_next_diff(i, diffs, ann_text, config.tsek)

                    # adjust next diff[i+1] if it's first char in tsek
                    elif diffs[i+1][1][0] == config.tsek:
                        ann_text = adjust_next_diff(i, diffs, ann_text, config.tsek)

                    # adjust next diff[i+1] if it's first char in shed
                    elif diffs[i+1][1][0] == config.shed:
                        ann_text = adjust_next_diff(i, diffs, ann_text, config.shed)

                    # adjust next diff[i+1] if its first char is line return
                    elif diffs[i+1][1][0] == '\n':
                        ann_text = adjust_next_diff(i, diffs, ann_text, '\n')

                if len(diffs[i][1]) == 1:
                    ann_text += diffs[i][1]
                else:
                    ann_text += '$'
        else:
            if verbose: print(diffs[i])
            ann_text += diffs[i][1]
    double_tsek_idxs = parse_double_tsek(ann_text)
    print(f'\t- Transferred {len(double_tsek_idxs)} out of {dest_text.count(config.double_tsek_sym)}')
    return double_tsek_idxs, ann_text

In [13]:
#NSM case
p_text = 'དེ་ནས་ཀོ$ཧཱུཾ་བཱི་ན་'
d_text = 'དེ་ནས་ཀཽ་ཤཱཾ་བཱི་ན་གནས་'
transfer_dtsek(d_text, p_text)

	- Transferred 1 out of 1


([9], 'དེ་ནས་ཀཽ་$ཤཱཾ་བཱི་ན་གནས་')

In [14]:
# adjust marker which is diffed before tsek
p_text = 'རིག་པས$༔མ་རིག་པའི་སྒོ་'
d_text = 'རིག་པས་མ་རིག་པའི་སྒོ་'
transfer_dtsek(d_text, p_text)

	- Transferred 1 out of 1


([7], 'རིག་པས་$མ་རིག་པའི་སྒོ་')

In [15]:
p_text = 'ནི༑$ལྷ་སྦྱིན་འདི་ཉིད་'
d_text = '་ནི། ལྷ་སྦྱིན་འདི་ཉིད་'
transfer_dtsek(d_text, p_text)

	- Transferred 1 out of 1


([5], '་ནི། $ལྷ་སྦྱིན་འདི་ཉིད་')

In [16]:
p_text = 'ཟད་དེ། $ད་ནི་དེ་'
d_text = 'ཟད་དེ།\nད་ནི་དེ་'
transfer_dtsek(d_text, p_text)

	- Transferred 1 out of 1


([7], 'ཟད་དེ།\n$ད་ནི་དེ་')

In [17]:
p_text = 'ང་ཚིps$ལ། །'
d_text = 'ང་ཚིལ། །'
transfer_dtsek(d_text, p_text, verbose=True)

[(0, 'ང་ཚི'), (-1, 'ps$'), (0, 'ལ། །')]
(0, 'ང་ཚི')
(0, 'ལ། །')
	- Transferred 1 out of 1


([4], 'ང་ཚི$ལ། །')

In [18]:
pedurma_ann_text = 'ལས་བརྒྱ་ཐམ་པ་པ། ༄༅། །རྒྱ་གར་སྐད་དུ། ཀརྨ་ཤ་ཏ་ཀ། བོད་སྐད་དུ། ལས་བརྒྱ་ཐམ་པ་པོ། བམ་པོ་དང་པོ། ཐམས་ཅད་མཁྱེན་པ་ལ་ཕྱག་འཚལ་ལོ། །གང་ལས་འཇིག་རྟེན་བླ་མ་བདེ་གཤེགས་ཐོས་པའི་སྒོ་ནས་རབ་སྙན་བརྟན་པའི་གསུང་་་་ལྡན་གྱིས། །སེམས་ཅན་རྣམས་ལ་ཕན་པ་འབའ་ཞིག་བཞེད་ཕྱིར་བཤད་པ་རྣམ་པ་སྣ་ཚོགས་རང་ཉིད་ཀྱིས། །ལོག་པར་ལྟ་བའི་མུན་ནག་ཆེན་པོ་ཐིབས་པོར་$འཐོམས་ཤིངའཁྲུགས་པ་རྣམས་ལ་རབ་གསུངས་པ། །$དེ་ཡི་མིང་ནི་ལས་རྣམ་བརྒྱ་པ་ཞེས་བྱ་ཡོངས་སུ་ཚང་བ་བདག་གིས་བཤད་ཀྱིས་ཉོན། །སྤྱི་སྡོམ་ནི༑ཁྱི་མོ་དང་ནི་ཤིང་རྟ་དང་། །ཀ་ཙང་ཀ་ལ་བྱམས་མི་སྡུག། བྱ་དང་འཕྱེ་བོ་གང་པོ་དང་། །བུ་རྣམས་དང་ནི་བརྒྱ་བྱིན་ནོ། །སྡོམ་ནི།'
derge_text = '༄༅༅། །རྒྱ་གར་སྐད་དུ། ཀརྨ་ཤ་ཏ་ཀ། བོད་སྐད་དུ། ལས་བརྒྱ་ཐམ་པ་པ། བམ་པོ་དང་པོ། ཐམས་ཅད་མཁྱེན་པ་ལ་ཕྱག་འཚལ་ལོ། །གང་ལས་འཇིག་རྟེན་བླ་མ་བདེ་གཤེགས་ཐོས་པའི་སྒོ་ནས་རབ་སྙན་བརྟན་པའི་གསུང་ལྡན་གྱིས། །སེམས་ཅན་རྣམས་ལ་ཕན་པ་འབའ་ཞིག་བཞེད་ཕྱིར་བཤད་པ་རྣམ་པ་སྣ་ཚོགས་རང་ཉིད་ཀྱིས། །ལོག་པར་ལྟ་བའི་མུན་ནག་ཆེན་པོ་ཐིབས་པོར་འཐོམས་ཤིང་འཁྲུགས་པ་རྣམས་ལ་རབ་གསུངས་པ། །དེ་ཡི་མིང་ནི་ལས་རྣམ་བརྒྱ་པ་ཞེས་བྱ་ཡོངས་སུ་ཚང་བ་བདག་གིས་བཤད་ཀྱིས་ཉོན། །སྤྱི་སྡོམ་ནི། ཁྱི་མོ་དང་ནི་ཤིང་རྟ་དང་། །ཀ་ཙང་ཀ་ལ་བྱམས་མི་སྡུག །བྱ་དང་འཕྱེ་བོ་གང་པོ་དང་། །བུ་རྣམས་དང་ནི་བརྒྱ་བྱིན་ནོ། །སྡོམ་ནི། ཁྱི་མོ་མིག་ཆུང་'
transfer_dtsek(derge_text, pedurma_ann_text)

	- Transferred 2 out of 2


([290, 329],
 '༄༅༅། །རྒྱ་གར་སྐད་དུ། ཀརྨ་ཤ་ཏ་ཀ། བོད་སྐད་དུ། ལས་བརྒྱ་ཐམ་པ་པ། བམ་པོ་དང་པོ། ཐམས་ཅད་མཁྱེན་པ་ལ་ཕྱག་འཚལ་ལོ། །གང་ལས་འཇིག་རྟེན་བླ་མ་བདེ་གཤེགས་ཐོས་པའི་སྒོ་ནས་རབ་སྙན་བརྟན་པའི་གསུང་ལྡན་གྱིས། །སེམས་ཅན་རྣམས་ལ་ཕན་པ་འབའ་ཞིག་བཞེད་ཕྱིར་བཤད་པ་རྣམ་པ་སྣ་ཚོགས་རང་ཉིད་ཀྱིས། །ལོག་པར་ལྟ་བའི་མུན་ནག་ཆེན་པོ་ཐིབས་པོར་$འཐོམས་ཤིང་འཁྲུགས་པ་རྣམས་ལ་རབ་གསུངས་པ། །$དེ་ཡི་མིང་ནི་ལས་རྣམ་བརྒྱ་པ་ཞེས་བྱ་ཡོངས་སུ་ཚང་བ་བདག་གིས་བཤད་ཀྱིས་ཉོན། །སྤྱི་སྡོམ་ནི། ཁྱི་མོ་དང་ནི་ཤིང་རྟ་དང་། །ཀ་ཙང་ཀ་ལ་བྱམས་མི་སྡུག །བྱ་དང་འཕྱེ་བོ་གང་པོ་དང་། །བུ་རྣམས་དང་ནི་བརྒྱ་བྱིན་ནོ། །སྡོམ་ནི། ཁྱི་མོ་མིག་ཆུང་')

In [72]:
#export
def _run(fns):
    for pedurma_base_fn, ocr_dtsek_fn in fns:
        pedurma_dtsek_dir = ocr_dtsek_fn.parent.parent / "pedurma_dtseks"
        pedurma_dtsek_fn = pedurma_dtsek_dir / pedurma_base_fn.name
        print("Transfering ", pedurma_dtsek_fn.stem)
        _, pedurma_dtsek = transfer_dtsek(pedurma_base_fn.read_text(), ocr_dtsek_fn.read_text())
        pedurma_dtsek_fn.write_text(pedurma_dtsek)
            
def transfer_dtseks_to_pedurma(replace=False):
    def _filter_completed(fns):
        result = []
        for (pedurma_base_fn, ocr_dtsek_fn) in fns:
            pedurma_dtsek_fn = pedurma_dtsek_dir / pedurma_base_fn.name
            if pedurma_dtsek_fn.is_file():
                print(pedurma_dtsek_fn.stem, " completed")
                continue
            result.append((pedurma_base_fn, ocr_dtsek_fn))
        return result
    
    
    "Returns pedurma base and ocr dobule tsek"
    pedurma_base_dir = config.op_pechas_path / config.p_pecha_id / f"{config.p_pecha_id}.opf" / "base"
    pecha_base_fns = sorted([fn for fn in pedurma_base_dir.iterdir() if config.name == "tengyur" and fn.stem == "v052"])
    ocr_dtsek_fns = sorted([fn for fn in config.pedurma_output_path.iterdir() if fn.is_file()])
    
    pedurma_dtsek_dir = config.pedurma_output_path.parent / "pedurma_dtseks"
    pedurma_dtsek_dir.mkdir(exist_ok=True, parents=True)
    fns = zip(pecha_base_fns, ocr_dtsek_fns)
    if not replace: fns = _filter_completed(fns)
    _run(fns)

In [69]:
transfer_dtseks_to_pedurma()

[(Path('/home/tenzin/.openpecha/pechas/P000791/P000791.opf/base/v052.txt'), Path('output/tengyur/pedurma/I1PD95846.txt'))]


In [30]:
#export
def post_process():
    def _cleanup_dtseks(text):
        text = vol_fn.read_text()
        text = text.replace('$:', ':')
        text = text.replace(':$', ':')
        text = text.replace('$', ':')
        text = text.replace('::', ':')
        return text
    
    pedurma_dtsek_dir = config.pedurma_output_path.parent / "pedurma_dtseks"
    for vol_fn in pedurma_dtsek_dir.iterdir():
        print(f"post processing {vol_fn.stem} ...")
        text = vol_fn.read_text()
        text = _cleanup_dtseks(text)
        vol_fn.write_text(text)

In [28]:
post_process()

post processing v041 ...
post processing v014 ...
post processing v032 ...
post processing v047 ...
post processing v037 ...
post processing v033 ...
post processing v048 ...
post processing v018 ...
post processing v003 ...
post processing v017 ...
post processing v020 ...
post processing v016 ...
post processing v023 ...
post processing v034 ...
post processing v012 ...
post processing v008 ...
post processing v027 ...
post processing v051 ...
post processing v042 ...
post processing v050 ...
post processing v011 ...
post processing v013 ...
post processing v022 ...
post processing v029 ...
post processing v026 ...
post processing v021 ...
post processing v025 ...
post processing v005 ...
post processing v035 ...
post processing v039 ...
post processing v031 ...
post processing v010 ...
post processing v045 ...
post processing v044 ...
post processing v036 ...
post processing v007 ...
post processing v009 ...
post processing v024 ...
post processing v038 ...
post processing v028 ...


In [40]:
#export
def main():
    print("[INFO] building pedurma vols ...")
    build_pedurma_vols()
    print("[INFO] transfering dtesks to pedurma ...")
    transfer_dtseks_to_pedurma()
    print("[INFO] post processing ...")
    post_process()

In [32]:
#export
if __name__ == "__main__":
    if len(sys.argv) > 1 and sys.argv[1] == "k":
        config = KangyurConfig()
    else: 
        config = TengyurConfig()
    main()

NameError: name 'sys' is not defined